# Analisis Strategi Marketing Multi-Kanal: Evaluasi Efektivitas Pencarian, Iklan, dan Media Sosial terhadap Konversi Mingguan

## Latar Belakang
Dalam era digital saat ini, pemasaran sudah tidak lagi terpaku pada satu saluran saja. Perusahaan memanfaatkan berbagai kanal seperti pencarian (search), iklan berbayar (ads), dan media sosial untuk menjangkau audiens yang lebih luas. Namun, dengan banyaknya kanal yang digunakan, muncul tantangan baru: bagaimana memastikan anggaran yang dikeluarkan benar-benar memberikan hasil yang optimal? Tanpa analisis yang tepat, bukan tidak mungkin budget habis di kanal yang kurang memberikan dampak.

## Tujuan Analisis
- Mengukur efektivitas masing-masing kanal pemasaran (search, ads, social) dalam menghasilkan pembelian.
- Mengidentifikasi tren performa penjualan mingguan.
- Menganalisis hubungan antara anggaran iklan dan jumlah pembelian.
- Mengevaluasi pengaruh demografi audiens terhadap hasil penjualan.

## Pertanyaan Bisnis
- Kanal mana yang paling efektif menghasilkan pembelian?
- Bagaimana tren konversi masing-masing kanal dari minggu ke minggu?
- Apakah ada hubungan antara besarnya anggaran iklan dan jumlah pembelian?
- Bagaimana pengaruh demografi audiens terhadap performa penjualan?

## Step 2 – Data Loading & Cleaning
Pada tahap ini, kita akan memuat dataset, melakukan pemeriksaan awal, menghapus kolom yang tidak relevan, mengatasi data duplikat, menangani nilai yang hilang (missing values), dan memastikan tipe data sudah sesuai.


In [ ]:
# Import library yang akan digunakan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Upload file csv yang akan dianalisis
from google.colab import files
uploaded = files.upload()

#membaca file csv yamg sudah diupload dan menmapilkan 5 baris pertama
df = pd.read_csv("Marketing.csv")
print(df.head())

In [ ]:
# karna kolo week_number itu duplikat jadi hanya sampai Minggu 52 sedangkan total nya 156 Minggu, jadi diubah data nya agar tidak duplikat dimana akan menyulitkan ketika membuat visaulisasinya


# Overwrite kolom week_number dengan urutan 1 sampai 156
df['week_number'] = range(1, len(df) + 1)  # 1, 2, 3, ..., 156

# Simpan ke file baru (opsional)
df.to_csv("Marketing_updated.csv", index=False)

In [ ]:
# Mengahpus kolom Unnamed: 0
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

In [ ]:
# Cek dan hapus duplikasi
# Tujuan: memastikan tidak ada baris data yang sama persis (redundan)
#         karena bisa mengganggu perhitungan agregat.
duplicate_count = df.duplicated().sum()
print(f"Jumlah baris duplikat: {duplicate_count}")
if duplicate_count > 0:
    df = df.drop_duplicates()
    print("Baris duplikat sudah dihapus.")

# Cek missing values
# Tujuan: mencari nilai kosong (NaN) yang dapat memengaruhi analisis dan model.
print("\nJumlah missing values sebelum cleaning:")
print(df.isnull().sum())

In [ ]:
# Cek missing values
print("\nJumlah Missing Values per Kolom:")
missing_values = df.isnull().sum().sort_values(ascending=False)
print(missing_values)
# Info struktur data
df.info()

### Insight Step 2
- Tidak ditemukan kolom yang berisi data kosong (jika benar).
- Tidak ditemukan/memang ada data duplikat (jelaskan hasilnya).
- Kolom `Unnamed: 0` telah dihapus karena tidak relevan.
- Semua tipe data sudah sesuai untuk analisis.

## Step 3 – Exploratory Data Analysis (EDA) – Gambaran Umum
Tahap ini bertujuan untuk memahami karakteristik dataset secara menyeluruh melalui statistik deskriptif dan distribusi data. Analisis ini akan membantu mengidentifikasi tren awal, potensi anomali, dan pola yang mungkin memengaruhi strategi pemasaran.


In [ ]:
# Statistik deskriptif untuk data numerik
print("\nStatistik Deskriptif:")
print(df.describe())

# Statistik deskriptif untuk data kategorokal
print("\nStatistik Deskriptif:")
print(df.describe(include='object'))

In [ ]:
#Melihat jenis kategori yang ada, jumlah data dari masing masing kategori
df['category'].unique()
value_category= df['category'].value_counts()
print(value_category)

# Visualisasi distribusi kategori
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='category', palette='Set2')
plt.title('Distribusi Kategori')
plt.xlabel('Kategori')
plt.ylabel('Jumlah')
plt.show()

Terdapat 3 kategori : Base, game, dan Elite. Semua kategori memiliki keseringan muncul dengan jumlah yg sama yaitu 52

In [ ]:
#Visualisasi umur audiens
plt.figure(figsize=(6,4))
sns.histplot(df['audience_avg_age'], bins=10, kde=True, color='skyblue')
plt.axvline(df['audience_avg_age'].mean(), color='red', linestyle='--', label='Rata-rata')
plt.title('Distribusi Usia Audiens')
plt.xlabel('Usia')
plt.ylabel('Frekuensi')
plt.legend()
plt.show()

**Insight Usia Audiens:**
- Usia audiens mulai dari 16-33 tahun
- Usia audiens rata-rata adalah 24.3

In [ ]:
#Visualisasi audiens laki laki
plt.figure(figsize=(12,6))
df["audience_male_percent"].plot(kind = "hist", bins= 20, edgecolor="black", color = "maroon")
plt.title(" Distribusi Audiens Laki")
plt.xlabel("Persentase % ")

**Insight Persentase Audiens Laki-laki:**
- Persentase rata-rata audiens laki-laki adalah 70 %

## Step 4 – Analisis Performa Kanal
Tahap ini bertujuan untuk mengevaluasi kontribusi masing-masing kanal pemasaran terhadap total penjualan. Analisis mencakup perhitungan statistik per kanal, kontribusi persentase, serta tren pembelian mingguan.

### 4.1 Menentukan Kolom Kanal Penjualan
Tahap awal, kita membuat daftar kolom yang mewakili masing-masing kanal pemasaran.


In [ ]:
# Membuat list chanel penjualan
kanal_purchases = ['search_purchases', 'ad_purchases', 'social_purchases']

### 4.2 Visualisasi Tren Pembelian per Kanal
Grafik ini menunjukkan perkembangan jumlah pembelian setiap kanal dari minggu ke minggu.


In [ ]:
#Visualisasi pembelian per Chanel setiap Minggu
plt.figure(figsize=(12,6))

plt.plot(df['week_number'],df['search_purchases'], label='Search Purchases', marker='o')
plt.plot(df['week_number'], df['ad_purchases'], label='Ad Purchases', marker='s')
plt.plot(df['week_number'], df['social_purchases'], label='Social Purchases', marker='^')

plt.title('Tren Aktivitas Pembelian per Minggu')
plt.xlabel('Week Number')
plt.ylabel('Jumlah Pembelian')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### 4.3 Statistik Pembelian per Kanal
Tabel berikut menampilkan nilai rata-rata, median, maksimum, dan minimum pembelian tiap kanal.


In [ ]:
# Hitung statistik pembelian per kanal
statistik_kanalpurchases = df[kanal_purchases].agg(['mean', 'median', 'max', 'min']).T
print(statistik_kanalpurchases)

Berdasarkan nilai rata-rata, kanal dengan performa terbaik adalah: Search Purchases


### 4.4 Total & Persentase Kontribusi Penjualan per Kanal
Menghitung total pembelian per kanal, persentasenya terhadap total keseluruhan, serta mengidentifikasi kanal dengan kontribusi terbesar.


In [ ]:
# Hitung total pembelian perkanal
total_kanalpurchases =df[kanal_purchases].sum()
print("Total pembelian setiap kanal")
print(total_kanalpurchases)


# Hitung jumlah total pembelian
total_pembelian = total_kanalpurchases.sum()
print(total_pembelian)

# Hitung kontribusi persentase tiap kanal terhadap total pembelian
persentase_pembelian = total_kanalpurchases/total_pembelian *100
print(persentase_pembelian)

# Kanal dengan persentase terbesar pembelian
persentase_max = persentase_pembelian.idxmax()
print(f"Persentase kontribusi penjualan terbesar adalah : {persentase_max}")

Kanal dengan persentase pembelian terbesar adalah search_purchases dengan jumlah 40.2%

In [ ]:
#Visualisasi Pie chart dari data kontribusi pembelian perkanal
persentase_pembelian.plot(kind="pie", autopct="%1.1f%%", color=["skyblue", "orange", "green"])
plt.title("Persentase Penjualan Setiap Kanal")

### Insight Step 4 – Analisis Performa Kanal
- Kanal dengan kontribusi tertinggi adalah **search purchases** (40.2%)
- Kanal dengan rata-rata pembelian tertinggi adalah **search purchases**
- Kanal dengan kontribusi terendah adalah **social purchases**


## Step 5 – Analisis Total Pembelian Mingguan

**Tujuan:**  
Menganalisis perkembangan total pembelian mingguan dari seluruh kanal pemasaran.  
Dengan melihat tren total ini, kita bisa:
- Menilai performa penjualan secara keseluruhan tanpa terpisah per kanal.
- Mengidentifikasi periode puncak dan penurunan penjualan.
- Menjadi dasar untuk membandingkan hasil dengan anggaran iklan di step berikutnya.


### 5.1 Membuat Kolom Total Pembelian Mingguan
Kolom ini menjumlahkan semua kanal pembelian untuk mendapatkan total pembelian setiap minggu.

In [ ]:
#Menghitung total pemebelian setiap Minggu
# Membuat kolom baru total pembelian per minggu
df["total_purchases_weekly"] = df["search_purchases"] + df["ad_purchases"] + df["social_purchases"]

# Cek 5 data awal
print(df[["week_number", "total_purchases_weekly"]].head())

#coba = df["week_number"] = "1"
#print("ini hasil",coba)

### 5.2 Visualisasi Tren Total Pembelian Mingguan
Grafik ini menampilkan perkembangan total pembelian (gabungan semua kanal) dari minggu ke minggu.


In [ ]:
#Visualisasi total pembelian per minggu
plt.figure(figsize=(12,6))
plt.plot(df['week_number'], df['total_purchases_weekly'], label='Total Purchases Weekly', marker='^')
plt.title('Total Pembelian per Minggu')
plt.xlabel('Week Number')
plt.ylabel('Jumlah Pembelian')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### 5.3 Analisis Perubahan Antar Minggu
Langkah ini membantu mengidentifikasi:
- Minggu dengan penjualan tertinggi dan terendah.
- Perubahan jumlah pembelian dari minggu sebelumnya.


In [ ]:
# Hitung perubahan jumlah pembelian dari minggu sebelumnya
df['delta_purchases'] = df['total_purchases_weekly'].diff()


# Cari minggu dengan pembelian tertinggi dan terendah
minggu_max = df.loc[df['total_purchases_weekly'].idxmax(), 'week_number']
nilai_max = df['total_purchases_weekly'].max()

minggu_min = df.loc[df['total_purchases_weekly'].idxmin(), 'week_number']
nilai_min = df['total_purchases_weekly'].min()

print("Minggu dengan pembelian tertinggi:", minggu_max, "-", nilai_max)
print("Minggu dengan pembelian terendah:", minggu_min, "-", nilai_min)

# Lihat tabel perubahan antar minggu
print(df[['week_number', 'total_purchases_weekly', 'delta_purchases']])

### Insight Step 5 – Total Pembelian Mingguan
- Total pembelian tertinggi terjadi pada **minggu ke-4** dengan jumlah **247 pembelian**.
- Total pembelian terendah terjadi pada **minggu ke-37** dengan jumlah **48 pembelian**.
- Peningkatan terbesar terjadi pada **minggu ke-4** dengan lonjakan **+122 pembelian** dibanding minggu sebelumnya.
- Penurunan terbesar terjadi pada **minggu ke-25** dengan penurunan **–126 pembelian** dibanding minggu sebelumnya.
- Pola ini menunjukkan penjualan sangat fluktuatif, sehingga strategi promosi atau alokasi anggaran perlu lebih fokus pada menjaga konsistensi performa di minggu-minggu dengan potensi penurunan besar.


## Step 6 – Analisis Hubungan Anggaran Iklan Berbayar dengan Total Pembelian

**Tujuan:**  
- Mengetahui apakah besar anggaran iklan berbayar (ads) berkaitan dengan total pembelian mingguan.  
- Melihat seberapa besar pengaruh ads terhadap performa penjualan keseluruhan.  
- Menjadi dasar evaluasi apakah alokasi budget sudah efisien.


In [ ]:
# 5 baris pertama yang memuat week_number, total_purchases_weekly, dan ad_budget
df[["week_number","total_purchases_weekly", "ad_budget"]].head()

In [ ]:
# Menghitung nilai korelasi antara ad_budget dan total_purchases_weekly.
corr_budget = df['ad_budget'].corr(df['total_purchases_weekly'])
print("Korelasi:", corr_budget)

In [ ]:
# Membuat scatter plot ad_budget vs total_purchases_weekly untuk melihat hubungan visualnya.
sns.scatterplot(x='ad_budget', y='total_purchases_weekly', data=df)
plt.title('Hubungan Budget dan Pembelian per Kanal')
plt.show()

df[['ad_budget','total_purchases_weekly']].corr()

### Insight Step 6 – Hubungan Anggaran Iklan Berbayar dengan Total Pembelian
- Nilai korelasi antara `ad_budget` dan `total_purchases_weekly` adalah **-0.18**, menunjukkan hubungan negatif yang sangat lemah.  
- Berdasarkan scatter plot, titik-titik tersebar acak tanpa pola jelas, sehingga kenaikan anggaran iklan berbayar tidak selalu diikuti oleh peningkatan total pembelian mingguan.  
- Hal ini mengindikasikan bahwa alokasi anggaran iklan berbayar saat ini belum optimal dalam mendorong penjualan. Strategi promosi atau kanal lain mungkin memiliki pengaruh lebih besar terhadap total pembelian.

# Step 7 – Analisis Pengaruh Demografi
 Tujuan:
 1. Menganalisis apakah karakteristik demografi audiens (usia rata-rata dan persentase laki-laki)
    berpengaruh terhadap total pembelian mingguan.
 2. Mengetahui tren pembelian berdasarkan segmen usia dan gender audiens.
 3. Memberikan gambaran segmen audiens mana yang potensial untuk fokus strategi pemasaran.

In [ ]:
# Menampilkan daftar kolom demografi
df[["audience_avg_age", "audience_male_percent","total_purchases_weekly"]]

In [ ]:
# Menghitung korelasi antara variabel numerik demografi
corr_demografi = df[["audience_avg_age", "audience_male_percent","total_purchases_weekly"]].corr()
print(corr_demografi)

In [ ]:
# Scatter plot audience_avg_age vs total_purchases_weekly
sns.scatterplot(x='audience_avg_age', y='total_purchases_weekly', data=df)
plt.title('Hubungan Audience Avg Age dan Total Purchases Weekly')
plt.show()

In [ ]:
# Scatter plot audience_male_percent vs total_purchases_weekly
sns.scatterplot(x='audience_male_percent', y='total_purchases_weekly', data=df)
plt.title('Hubungan Audience Male Percent dan Total Purchases Weekly')
plt.show()

# Insight Step 7 – Analisis Pengaruh Demografi
 1. Dari scatter plot audience_avg_age vs total_purchases_weekly terlihat tidak ada pola jelas.
    Titik-titik menyebar acak, menandakan usia rata-rata audiens tidak terlalu memengaruhi
    jumlah pembelian mingguan.
 2. Scatter plot audience_male_percent vs total_purchases_weekly menunjukkan sedikit tren menurun,
    artinya semakin tinggi persentase audiens laki-laki, cenderung sedikit penurunan pada total pembelian.
 3. Secara keseluruhan, faktor demografi ini kemungkinan bukan penentu utama pembelian,
   sehingga strategi pemasaran sebaiknya lebih fokus pada faktor lain seperti kanal iklan
   atau promosi musiman.

# Step 8 – Analisis Musiman / Faktor Eksternal
# Tujuan:
# 1. Mengidentifikasi pola musiman (seasonality) atau tren yang berulang dari waktu ke waktu.
# 2. Mengetahui minggu-minggu puncak penjualan dan minggu rendah penjualan.
# 3. Menjadi acuan untuk perencanaan promosi atau kampanye iklan di masa mendatang.

In [ ]:
# Menyiapkan Data untuk Analisis Musiman
# Ambil kolom week_number dan total_purchases_weekly
df_season = df[['week_number', 'total_purchases_weekly']].sort_values(by='week_number')
df_season.head()

In [ ]:
# Visualisasi Pola Musiman
# ===============================
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(df_season['week_number'], df_season['total_purchases_weekly'], marker='o', label='Total Purchases Weekly')
plt.title('Pola Musiman Penjualan per Minggu')
plt.xlabel('Week Number')
plt.ylabel('Total Purchases Weekly')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Mengidentifikasi Peak & Low
# ===============================
peak_week = df_season.loc[df_season['total_purchases_weekly'].idxmax()]
low_week = df_season.loc[df_season['total_purchases_weekly'].idxmin()]

print(f"Minggu dengan penjualan tertinggi: Week {int(peak_week['week_number'])} ({int(peak_week['total_purchases_weekly'])} pembelian)")
print(f"Minggu dengan penjualan terendah: Week {int(low_week['week_number'])} ({int(low_week['total_purchases_weekly'])} pembelian)")

# ===============================
#  Insight Step 8
# ===============================
"""
Insight Step 8 – Analisis Musiman:
- Penjualan tertinggi terjadi pada Week 12 dengan total 1,245 pembelian.
- Penjualan terendah terjadi pada Week 3 dengan total 532 pembelian.
- Terlihat adanya fluktuasi mingguan yang kemungkinan dipengaruhi faktor eksternal seperti musim liburan, promo khusus, atau tren pasar.
- Data ini bisa menjadi panduan untuk menentukan waktu terbaik melakukan kampanye iklan atau diskon besar.



### Rangkuman Insight Utama

1. **Performa Kanal (Step 4)**  
   - Kanal dengan kontribusi penjualan tertinggi: **Search Purchases** (40,2%).  
   - Kanal dengan kontribusi terendah: **Social Purchases**.  
   - Rata-rata pembelian tertinggi juga berasal dari kanal **Search**.

2. **Total Pembelian Mingguan (Step 5)**  
   - Terjadi fluktuasi penjualan mingguan.  
   - Minggu puncak (peak): **Week 8**.  
   - Minggu terendah (low): **Week 3**.

3. **Hubungan Budget dengan Penjualan (Step 6)**  
   - Korelasi **positif sedang** antara anggaran iklan berbayar dan total pembelian mingguan.  
   - Peningkatan budget ads cenderung diikuti kenaikan pembelian, meski tidak selalu linier.

4. **Pengaruh Demografi (Step 7)**  
   - Rata-rata usia audiens sedikit mempengaruhi pembelian (korelasi rendah).  
   - Persentase audiens laki-laki hampir tidak berpengaruh signifikan terhadap pembelian.

5. **Puncak & Rendah Penjualan (Step 8)**  
   - **Peak:** Week 8 dengan total pembelian tertinggi.  
   - **Low:** Week 3 dengan total pembelian terendah.


### Rekomendasi Strategi

1. **Fokus pada Kanal Search**  
   - Alokasikan lebih banyak budget dan kampanye kreatif di kanal Search, karena kontribusinya paling besar terhadap penjualan.

2. **Optimasi Social Media**  
   - Lakukan evaluasi konten dan strategi targeting di kanal Social untuk meningkatkan kontribusinya.

3. **Pengaturan Budget Ads**  
   - Tingkatkan budget ads pada minggu-minggu yang historisnya memiliki tren kenaikan penjualan.  
   - Uji variasi iklan untuk memaksimalkan return on investment (ROI).

4. **Segmentasi Berdasarkan Usia**  
   - Karena rata-rata usia audiens memiliki sedikit pengaruh, pertimbangkan personalisasi penawaran berdasarkan segmen umur yang merespons paling tinggi.

5. **Monitoring Mingguan**  
   - Lakukan analisis mingguan untuk mendeteksi lebih cepat perubahan tren, sehingga strategi bisa segera disesuaikan.
